In [1]:
import bpy
import subprocess
import json
import os
import random
from pathlib import Path

## Getting the position of the armature from Blender

This function 
- reaches out to the specified blender file
- returns the position of the armature

In [9]:
def get_armature_from_blender(blend_file_path):
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender'     # Mac
    # blender_executable_path = ''     # Windows
    script_file_path = 'utils/get_armature.py'
    command = [
        blender_executable_path,
        '-b', blend_file_path,
        '-P', script_file_path,
        '--',
    ]    
    result = subprocess.run(command, capture_output=True, text=True)
    # print(result)
    output_lines = result.stdout.split('\n')
    json_data = ''
    recording = False
    for line in output_lines:
        if line == '---JSON_START---':
            recording = True
        elif line == '---JSON_END---':
            break
        elif recording:
            json_data += line

    try:
        if json_data:
            armature_data = json.loads(json_data)
            return armature_data
        else:
            print("No armature data was found.")
            return None
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON from Blender output: {e}")
        return None

## Applying the Perturbation

This function...
- changes the rig in a random way
- based on the constraints it is given in armature_boundaries.json
- the idle/base configuration is being taken from armature_data.json

In [10]:
def perturbate_armature_in_blender(blend_file_path):
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender' # Mac
    # blender_executable_path = '' # Windows    
    script_file_path = 'utils/perturbate_armature.py'  
    path_of_idle_armature = 'idle_armature.json'
    
    command = [
        blender_executable_path,
        '-b', blend_file_path,
        '--python', script_file_path,
        '--', path_of_idle_armature
    ]
    result = subprocess.run(command)
    print(result)

## Rendering the scene in Blender

In [2]:
def render_blend_file(blend_file_path, output_folder, out_file_name, running_number, current_rig_params):
    os.makedirs(output_folder, exist_ok=True)
    blender_executable_path = '/Applications/Blender3.4.1.app/Contents/MacOS/Blender' # Mac
    # blender_executable_path = '' # Windows
    output_file_path = os.path.join(output_folder, f"{running_number}_{out_file_name}")
    params_json_path = os.path.join(output_folder, 'rendering_rig_params.json')

    py_command = "import bpy; bpy.context.scene.render.resolution_x=512; bpy.context.scene.render.resolution_y=512; bpy.context.scene.render.resolution_percentage=100"

    command = [
        blender_executable_path,        # Path to the Blender executable
        '-b', blend_file_path,          # Specify the .blend file
        '--python-expr', py_command,    # Run the Python command to set resolution
        '-o', output_file_path,         # Set the output file path
        '-F', 'PNG',                    # Set the output format to PNG
        '-f', '1'                       # Render the first frame (change as needed)
    ]
    subprocess.run(command)    
    print(f"Render complete: {output_file_path}")

    if Path(params_json_path).exists():
        with open(params_json_path, 'r') as file:
            data = json.load(file)
    else:
        data = {}
    
    data[str(running_number)] = current_rig_params
    
    with open(params_json_path, 'w') as file:
        json.dump(data, file, indent=4) 

# Creating the Idle

When running the first time with a new character, first run this, so that the idle armature is being saved:

In [12]:
first_time = False
path_to_blender_file = '/Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend'

idle_rig_params = get_armature_from_blender(path_to_blender_file)
print("The idle position of the relevant bones in the armature is: ")
print(idle_rig_params)
if first_time:
    output_file='data/armature_configuration/idle_armature.json'
    with open(output_file, 'w') as outfile:
        json.dump(idle_rig_params, outfile, indent=4)
    print("Saved the idle armature to idle_armature.json")

The idle position of the relevant bones in the armature is: 
{'jaw_master': {'location': [0.07955741882324219, -7.746394634246826, 160.69256591796875], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'cheek.B.R.001': {'location': [-5.149785041809082, -11.945501327514648, 157.17555236816406], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'cheek.T.R.001': {'location': [-3.9348154067993164, -13.465989112854004, 159.122314453125], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'lips.R': {'location': [-2.4214324951171875, -13.92071533203125, 155.97479248046875], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'brow.T.R.003': {'location': [-1.4480360746383667, -14.290176391601562, 163.5772247314453], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'lid.B.R.002': {'location': [-3.8495543003082275, -13.380725860595703, 161.36753845214844], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'cheek.B.L.001': {'location': [5.308899879455566, -11.945501327514648, 157.17555236816406], 'rotation': [1.0, 0.0, 0.0, 0.0]}, 'cheek.T.L.001': {'location': [4.093930721282959, -13.46

Then let's render the idle picture and create 

In [ ]:
print(idle_rig_params)

In [6]:
# path_to_blender_file = '/Users/leonpaletta/Coding/MT/root/data/blender_characters/sad_ulf.blend'
# render_blend_file(blend_file_path = path_to_blender_file, output_folder = 'data/render_trainset', out_file_name = 'sad.png', running_number = 'idle_params', current_rig_params = idle_rig_params)

## Rendering Results of single Characters

In [22]:
path_to_blender_file = '/Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend'
out_name = 'neutral_ulf'
render_blend_file(blend_file_path = path_to_blender_file, output_folder = 'data/output_generation', out_file_name = out_name, running_number = '', current_rig_params = None)

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend
Fra:1 Mem:87.56M (Peak 88.69M) | Time:00:00.62 | Syncing Light
Fra:1 Mem:87.56M (Peak 88.69M) | Time:00:00.62 | Syncing Camera
Fra:1 Mem:87.57M (Peak 88.69M) | Time:00:00.62 | Syncing Ch22_Hair
Fra:1 Mem:120.76M (Peak 135.60M) | Time:00:01.13 | Syncing Ch22_Body
Fra:1 Mem:378.08M (Peak 632.81M) | Time:00:02.63 | Syncing rig
Fra:1 Mem:378.09M (Peak 632.81M) | Time:00:02.63 | Syncing Ch22_Eyelashes
Fra:1 Mem:378.17M (Peak 632.81M) | Time:00:02.63 | Syncing metarig
Fra:1 Mem:377.65M (Peak 632.81M) | Time:00:02.64 | Rendering 1 / 64 samples
Fra:1 Mem:376.01M (Peak 632.81M) | Time:00:02.74 | Rendering 26 / 64 samples
Fra:1 Mem:376.01M (Peak 632.81M) | Time:00:02.78 | Rendering 51 / 64 samples
Fra:1 Mem:376.01M (Peak 632.81M) | Time:00:02.80 | Rendering 64 / 64 s

### Data Generator
Creates samples...
- in the predefined range specified in armature_boundaries.json
- takes as idle position the information from idle_armature.json
- takes into consideration all the bones specified in relevant_armature.json 

In [24]:
n_samples = 200

In [25]:
for i in range(1, n_samples): 
    perturbate_armature_in_blender(path_to_blender_file)
    current_rig_params = get_armature_from_blender(path_to_blender_file)

    diff_rig_params = {}
    for part, params in current_rig_params.items():
        idle_part_params = idle_rig_params.get(part, {})
        diff_location = [curr - idle for curr, idle in zip(params['location'], idle_part_params.get('location', [0, 0, 0]))]
        diff_rotation = [curr - idle for curr, idle in zip(params['rotation'], idle_part_params.get('rotation', [1, 0, 0, 0]))]
        diff_rig_params[part] = {'location': diff_location, 'rotation': diff_rotation}

    
    render_blend_file(blend_file_path = path_to_blender_file, output_folder = 'data/render_trainset', out_file_name = 'rendis', running_number = str(i), current_rig_params = diff_rig_params)

Reloading external rigs...
Reloading external metarigs...
Info: Saved "ulf.blend"
Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend

Blender quit
CompletedProcess(args=['/Applications/Blender3.4.1.app/Contents/MacOS/Blender', '-b', '/Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend', '--python', 'utils/perturbate_armature.py', '--', 'idle_armature.json'], returncode=0)
Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:48:52)
Read prefs: /Users/leonpaletta/Library/Application Support/Blender/3.4/config/userpref.blend
Read blend: /Users/leonpaletta/Coding/MT/root/data/blender_characters/ulf.blend
Fra:1 Mem:87.56M (Peak 88.69M) | Time:00:00.60 | Syncing Light
Fra:1 Mem:87.56M (Peak 88.69M) | Time:00:00.60 | Syncing Camera
Fra:1 Mem:87.57M (Peak 88.69M) | Time:00:00.60 | Syncing Ch22_Hair
